In [11]:
import chromadb 
import os 
import json
import requests 
from dotenv import load_dotenv
from datetime import datetime, timedelta, timezone
import asyncio
from newspaper import Article
import nltk

# Download required NLTK data
nltk.download('punkt_tab', quiet=True)

load_dotenv()

# create class to do newsprocessing
class newsprocessing:
    def __init__(self):
        self.news_api_key = os.getenv("NEWSAPI_KEY")
        self.vector_db = chromadb.Client()
        self.collection = self.vector_db.get_or_create_collection("market_news")
        self.all_summaries = []  # Track all summaries

    def load_tickers(self):
        return ["AAPL", "TSLA", "GOOG", "MSFT", "LUNR", "NVDA"]
    
    def fetch_news(self, ticker, start_date, end_date):
        url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&RANGE={start_date}&RANGE={end_date}&apikey={self.news_api_key}"
        response = requests.get(url)
        return response.json()

    async def fetch_and_process_news(self):
        tickers = self.load_tickers()
        today = datetime.now(timezone.utc).strftime("%Y-%m-%d")
        yesterday = (datetime.now(timezone.utc) - timedelta(days=1)).strftime("%Y-%m-%d")
        
        print(f" Starting news processing for {len(tickers)} tickers: {', '.join(tickers)}")
        print(f" Date range: {yesterday} to {today}")
        print("-" * 60)
        
        task = [self.process_tickers(ticker, yesterday, today) for ticker in tickers]
        await asyncio.gather(*task)

    async def process_tickers(self, ticker, start_date, end_date):
        print(f" Fetching news for {ticker}...")
        news_data = self.fetch_news(ticker, start_date, end_date)
        
        if not news_data.get("feed"):
            print(f" No news found for {ticker}")
            return
            
        articles = news_data["feed"]
        print(f" Found {len(articles)} articles for {ticker}")
        
        for i, article in enumerate(articles, 1):
            title = article.get("title", "")[:80] + "..." if len(article.get("title", "")) > 80 else article.get("title", "")
            print(f"   [{ticker}] Processing article {i}/{len(articles)}: {title}")
            
            cleaned_article = self.cleaning(article)
            await self.store_in_db(cleaned_article, ticker)
            
            # Add summary to our collection for the text file
            if cleaned_article['content']:
                summary_entry = f"[{ticker}] {cleaned_article['title']}: {cleaned_article['content']}"
                self.all_summaries.append(summary_entry)
            
            print(f" Stored article {i}/{len(articles)} for {ticker}")
        
        print(f"🎉 Completed processing {len(articles)} articles for {ticker}")
        print("-" * 40)

    def extract_article_content(self, url):
        """Extract article content using newspaper3k"""
        try:
            print(f" Processing with newspaper3k: {url[:60]}...")
            
            news = Article(url)
            news.download()
            news.parse()
            news.nlp()
            
            content = news.summary if news.summary else news.text[:1000]
            print(f" Extracted {len(content)} characters")
            
            return {
                'content': content,
                'full_text': news.text[:2000] if news.text else "",
                'authors': news.authors,
                'publish_date': news.publish_date
            }
            
        except Exception as e:
            print(f" newspaper3k extraction failed: {e}")
            return None
            
    def cleaning(self, article):
        # Get the full article content using newspaper3k
        article_url = article.get("url", "")
        extracted_data = self.extract_article_content(article_url) if article_url else None
        
        # Use extracted content or fallback to original summary
        content = extracted_data['content'] if extracted_data else article.get("summary", "")
        full_text = extracted_data['full_text'] if extracted_data else ""
        
        return {
            "title": article.get("title", ""),
            "summary": article.get("summary", ""),
            "content": content,
            "full_content": full_text,
            "sentiment": article.get("overall_sentiment_score", 0),
            "published": article.get("time_published", ""),
            "source": article.get("source", ""),
            "url": article_url,
            "authors": extracted_data['authors'] if extracted_data else [],
            "extracted_date": str(extracted_data['publish_date']) if extracted_data and extracted_data['publish_date'] else ""
        }
    
    async def store_in_db(self, article, ticker):
        embedding_text = f"{article['title']} {article['content']}"
        self.collection.add(
            documents=[embedding_text],
            metadatas=[
                {
                    "ticker": ticker,
                    "sentiment": article["sentiment"],
                    "published": article["published"],
                    "source": article["source"],
                    "url": article.get("url", ""),
                    "authors": str(article.get("authors", []))
                }
            ],
            ids=[f"{ticker}_{article['published']}_{hash(article['title'])}"]
        )
    
    def save_summaries_to_file(self):
        """Save all summaries to a text file"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"news_summary_{timestamp}.txt"
        
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(f"News Summaries - Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write("=" * 80 + "\n\n")
                
                for summary in self.all_summaries:
                    f.write(summary + "\n\n")
                    
            print(f" Saved {len(self.all_summaries)} summaries to {filename}")
            return filename
            
        except Exception as e:
            print(f" Error saving summaries: {e}")
            return None
    
async def main():
    start_time = datetime.now()
    print(f" Starting at {start_time.strftime('%H:%M:%S')}")
    
    process = newsprocessing()
    await process.fetch_and_process_news()
    
    # Save summaries to text file
    summary_file = process.save_summaries_to_file()
    
    end_time = datetime.now()
    duration = end_time - start_time
    print(f" News processing complete!")
    print(f" Total time: {duration}")
    if summary_file:
        print(f" Summary file: {summary_file}")

In [14]:
main()

<coroutine object main at 0x000001DF30837240>

In [2]:
news_data = []
with open("news_summary_20251010_192220.txt", "r", encoding="utf-8") as file:
    news_data = file.readlines()
    print(f"loaded {len(news_data)}")

loaded 1620


In [3]:
import ollama
embedding_model = "hf.co/CompendiumLabs/bge-base-en-v1.5-gguf"
language_model = "qwen3:latest" # qwen is goated going through 
 
vector_db = []

In [4]:
def add_chunk_to_database(chunk):
    embedding = ollama.embed(model=embedding_model, input=chunk)["embeddings"][0]
    vector_db.append((chunk, embedding))

In [6]:
for i, chunk in enumerate(news_data):
    add_chunk_to_database(chunk)
    print(f"added chunk {i+1}/{len(news_data)} to the database")

added chunk 1/1620 to the database
added chunk 2/1620 to the database
added chunk 3/1620 to the database
added chunk 4/1620 to the database
added chunk 5/1620 to the database
added chunk 6/1620 to the database
added chunk 7/1620 to the database
added chunk 8/1620 to the database
added chunk 9/1620 to the database
added chunk 10/1620 to the database
added chunk 11/1620 to the database
added chunk 12/1620 to the database
added chunk 13/1620 to the database
added chunk 14/1620 to the database
added chunk 15/1620 to the database
added chunk 16/1620 to the database
added chunk 17/1620 to the database
added chunk 18/1620 to the database
added chunk 19/1620 to the database
added chunk 20/1620 to the database
added chunk 21/1620 to the database
added chunk 22/1620 to the database
added chunk 23/1620 to the database
added chunk 24/1620 to the database
added chunk 25/1620 to the database
added chunk 26/1620 to the database
added chunk 27/1620 to the database
added chunk 28/1620 to the database
a

In [7]:
# use cosine similiarity to find close chunks 
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

In [8]:
def retrieve(query, top_n=10):
    query_embedding = ollama.embed(model=embedding_model, input=query)["embeddings"][0]
    similarities = []
    for chunk, embedding in vector_db:
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk, similarity))
    # sort by similarity in descending order, because higher similarity means more relevant chunks
    similarities.sort(key=lambda x: x[1], reverse=True)
    # finally, return the top N most relevant chunks
    return similarities[:top_n]

In [9]:
input_query = input("Ask me anything about stocks! ")
retrieved_knowledge = retrieve(input_query)

print("retrieved knowledge: ")
for chunk, similarity in retrieved_knowledge:
    print(f" - (similarity: {similarity:.2f}) {chunk}")

context_text = '\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])

instruction_prompt = f"""
You are a professional stock market analysis assistant with access to real-time news via RAG.
You must always ground your commentary in the retrieved articles.
IMPORTANT: Do NOT show your thinking process. Do NOT use <think> tags or explain your reasoning. Just give your final response directly.
Rules:
1. Use ONLY the retrieved news articles as your source of information.
2. Always rely on the LATEST news provided in the feed. Do not invent or fabricate numbers, prices, or events.
3. Respond in ONE well-structured paragraph that flows naturally, as if written for a financial news briefing.
4. No bullet points, no lists, no over-formatting.
5. The response should feel like a concise but insightful market recap.
6. Maintain a professional, analytical tone—clear, objective, and polished.
7. Integrate facts from the news seamlessly into the commentary, highlighting key drivers of market sentiment.
8. If the news is routine, frame it as stability or consolidation; if it is significant, highlight its impact on markets, sectors, or investor sentiment.
9. Do NOT speculate recklessly or use slang; remain focused on credible, news-based insights.
10. ALWAYS end by saying this is not financial advice do not trade off what I say. 
Your goal is to deliver a single, cohesive, professional market update that summarizes the latest news into a clear, authoritative narrative.
{context_text}
"""

retrieved knowledge: 
 - (similarity: 0.73) [TSLA] Tesla Q3 Deliveries Smash Estimates, But Wall Street Wasn't Impressed. What Gives?: Tesla recently reported third-quarter deliveries that came in well ahead of what Wall Street analysts expected.

 - (similarity: 0.72) Tesla's Affordable Trims Invite CriticismTesla Inc. (NASDAQ:TSLA) also announced new affordable trim levels for the EV giant's Model Y and Model 3 in the U.S.

 - (similarity: 0.71) [TSLA] Tesla's Shanghai factory deliveries hit record as Model Y variant woos buyers: Tesla’s Shanghai Gigafactory saw a jump in deliveries last month, as its upgraded models struck a chord with wealthy Chinese consumers who favour spacious vehicles.

 - (similarity: 0.70) [TSLA] Tesla's New Robotaxi-Ready Models Strengthen the Bull Case for the Stock: Cheaper vehicle trims with the latest self-driving hardware could widen Tesla's addressable market and feed its autonomy ambitions.

 - (similarity: 0.69) [TSLA] Tesla Cybercab Spotted At Gigaf

In [10]:
stream = ollama.chat(
    model=language_model,
    messages=[
        {'role': 'system', 'content': instruction_prompt},
        {'role': 'user', 'content': input_query},
    ],
    stream=True
)

print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Chatbot response:
<think>
Okay, let's see. The user is asking for the latest news on Tesla. I need to go through the provided articles and pick out the key points. First, there's the Q3 delivery numbers that beat estimates, but Wall Street wasn't impressed. That's a big point. Then there are the new affordable trims for Model Y and 3, which are getting some criticism. The Shanghai factory deliveries hit a record, especially with the upgraded Model Y. Also, the new robotaxi-ready models are mentioned as a bullish factor. The Cybercab testing in Texas and California is another point. The NHTSA investigation into FSD is causing some stock drops, but Gene Munster thinks it might be good for Tesla. Plus, Tesla needs PR staff to handle regulation. I should structure this into a coherent paragraph, making sure to highlight the main drivers like delivery numbers, new models, regulatory issues, and market reactions. Need to end with a disclaimer about not giving financial advice. Let me check i